In [1]:
import sqlite3
import pandas as pd
import os
import helper as h

/home/seth/.local/lib/python3.8/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


**Connect to Database**

In [2]:
# Create a SQL connection to our SQLite database
con = sqlite3.connect("data/owid_covid_data.sqlite3")
con.row_factory = sqlite3.Row # fetch both column names and value pairs
cur = con.cursor()

### Case Fatality

In [3]:
# case fatality ratio globally
df = h.execute_q('''
    WITH CaseFatality (Country, TotalCases, TotalDeaths) AS
    (
        SELECT location, MAX(total_cases), MAX(total_deaths)
        FROM CovidDeaths
        WHERE continent IS NOT NULL
        GROUP BY location
    ) 
    SELECT 
        SUM(TotalCases) AS TotalCases,
        SUM(TotalDeaths) AS TotalDeaths,
        (SUM(TotalDeaths)/SUM(TotalCases)) AS CaseFatalityRate
    FROM CaseFatality;
''', cur)
df.to_csv("df_output/case_fatality_ratio_global.csv", index=False)
df

,TotalCases,TotalDeaths,CaseFatalityRate
0,397259846.0,5750928.0,0.014476


In [21]:
# case fatality through time in each continent
df = h.execute_q('''
    SELECT det.location, det.date,
            MAX(det.total_cases) AS cuml_TotalCases,
            MAX(det.total_deaths) AS cuml_TotalDeaths,
            (MAX(det.total_deaths)/MAX(det.total_cases)) AS cuml_CaseFatalityRate
    FROM CovidDeaths det
    JOIN CovidVacc vac
        ON vac.iso_code == det.iso_code
        AND vac.date = det.date
    WHERE
        det.continent IS NULL
            AND det.location NOT LIKE '%income'
            AND det.location NOT IN ('World', 'International', 'European Union')
    GROUP BY det.location, det.date;
''', cur).fillna(0)
df.to_csv("df_output/case_fatality_by_time_continent.csv", index=False)
df

,location,date,cuml_TotalCases,cuml_TotalDeaths,cuml_CaseFatalityRate
0,Africa,2020-02-13,0.0,0.0,0.000000
1,Africa,2020-02-14,1.0,0.0,0.000000
2,Africa,2020-02-15,1.0,0.0,0.000000
3,Africa,2020-02-16,1.0,0.0,0.000000
4,Africa,2020-02-17,1.0,0.0,0.000000
...,...,...,...,...,...
4426,South America,2022-02-03,49644699.0,1221306.0,0.024601
4427,South America,2022-02-04,49990900.0,1223114.0,0.024467
4428,South America,2022-02-05,50262915.0,1224803.0,0.024368
4429,South America,2022-02-06,50409587.0,1225781.0,0.024316


### Vaccination, Infection, R values

In [22]:
# vaccinated percent and R value in each country
df = h.execute_q('''
    SELECT det.location, det.date,
            MAX(vac.total_vaccinations) AS cuml_TotalVaccinations,
            MAX(det.reproduction_rate) AS R_value, 
            (MAX(det.total_deaths)/MAX(det.total_cases))*100 AS cuml_CaseFatalityRate,
            MAX(vac.people_vaccinated/det.population)*100 AS cuml_VaccinatedPercent
    FROM CovidDeaths det
    JOIN CovidVacc vac
        ON vac.iso_code == det.iso_code
        AND vac.date = det.date
    WHERE
        det.continent IS NOT NULL
            AND det.location NOT LIKE '%income'
            AND det.location NOT IN ('World', 'International', 'European Union')
    GROUP BY det.location, det.date;
''', cur).fillna(0)
df.to_csv("df_output/vaccination&r_value.csv", index=False)
df

,location,date,cuml_TotalVaccinations,R_value,cuml_CaseFatalityRate,cuml_VaccinatedPercent
0,Afghanistan,2020-02-24,0.0,0.00,0.000000,0.000000
1,Afghanistan,2020-02-25,0.0,0.00,0.000000,0.000000
2,Afghanistan,2020-02-26,0.0,0.00,0.000000,0.000000
3,Afghanistan,2020-02-27,0.0,0.00,0.000000,0.000000
4,Afghanistan,2020-02-28,0.0,0.00,0.000000,0.000000
...,...,...,...,...,...,...
150614,Zimbabwe,2022-02-03,7600946.0,0.45,2.327410,28.400142
150615,Zimbabwe,2022-02-04,7611929.0,0.44,2.327410,28.438665
150616,Zimbabwe,2022-02-05,7617184.0,0.00,2.327237,28.455555
150617,Zimbabwe,2022-02-06,7620153.0,0.00,2.327237,28.464738


### Infection Rates

In [38]:
# how does the infection rate changes before and after vaccination
# consider num of tests as well cuz new tests could reval more infections
df = h.execute_q('''
    SELECT det.date, det.location, det.population, 
           det.new_cases, det.reproduction_rate, det.total_cases,
           vac.total_tests, vac.new_tests, vac.total_vaccinations
    FROM CovidDeaths det
    JOIN CovidVacc vac ON
        vac.iso_code == det.iso_code 
        AND vac.date == det.date
    WHERE det.continent IS NOT NULL
''', cur).fillna(0)
df.to_csv("df_output/vacc&infection_trends.csv", index=False)
df

,date,location,population,new_cases,reproduction_rate,total_cases,total_tests,new_tests,total_vaccinations
0,2020-02-24,Afghanistan,39835428.0,5.0,0.00,5.0,0.0,0.0,0.0
1,2020-02-25,Afghanistan,39835428.0,0.0,0.00,5.0,0.0,0.0,0.0
2,2020-02-26,Afghanistan,39835428.0,0.0,0.00,5.0,0.0,0.0,0.0
3,2020-02-27,Afghanistan,39835428.0,0.0,0.00,5.0,0.0,0.0,0.0
4,2020-02-28,Afghanistan,39835428.0,0.0,0.00,5.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
150614,2022-02-03,Zimbabwe,15092171.0,158.0,0.45,230170.0,1994516.0,0.0,7600946.0
150615,2022-02-04,Zimbabwe,15092171.0,0.0,0.44,230170.0,0.0,0.0,7611929.0
150616,2022-02-05,Zimbabwe,15092171.0,232.0,0.00,230402.0,2000510.0,0.0,7617184.0
150617,2022-02-06,Zimbabwe,15092171.0,0.0,0.00,230402.0,0.0,0.0,7620153.0


In [23]:
# how does the infection rate changes before and after vaccination
# consider num of tests as well cuz new tests could reval more infections
df = h.execute_q('''
    SELECT det.date, det.location, det.population, 
           det.new_cases, det.reproduction_rate, det.total_cases,
           vac.total_tests, vac.total_vaccinations
    FROM CovidDeaths det
    JOIN CovidVacc vac ON
        vac.iso_code == det.iso_code 
        AND vac.date == det.date
    WHERE det.continent IS NULL
        AND det.location NOT LIKE '%income'
        AND det.location NOT IN ('World', 'International', 'European Union');
''', cur).fillna(0)
df.to_csv("df_output/vacc&infection_trends.csv", index=False)
df

,date,location,population,new_cases,reproduction_rate,total_cases,total_tests,total_vaccinations
0,2020-02-13,Africa,1.373486e+09,0.0,0,0.0,0,0.0
1,2020-02-14,Africa,1.373486e+09,1.0,0,1.0,0,0.0
2,2020-02-15,Africa,1.373486e+09,0.0,0,1.0,0,0.0
3,2020-02-16,Africa,1.373486e+09,0.0,0,1.0,0,0.0
4,2020-02-17,Africa,1.373486e+09,0.0,0,1.0,0,0.0
...,...,...,...,...,...,...,...,...
4426,2022-02-03,South America,4.342601e+08,423496.0,0,49644699.0,0,724491504.0
4427,2022-02-04,South America,4.342601e+08,346201.0,0,49990900.0,0,728403628.0
4428,2022-02-05,South America,4.342601e+08,272015.0,0,50262915.0,0,729638029.0
4429,2022-02-06,South America,4.342601e+08,146672.0,0,50409587.0,0,730591048.0


In [7]:
# Total infection rate by location
df = h.execute_q('''
    SELECT location,
            MAX(Population) AS Population,
            MAX(total_cases) AS TotalCases, 
            (MAX(total_cases)/MAX(Population))*100 AS TotalInfectionRate
    FROM CovidDeaths
    WHERE continent IS NOT NULL
    GROUP BY location
    ORDER BY TotalInfectionRate DESC;
''', cur).fillna(0)
df.to_csv("df_output/total_infection_rate_by_location.csv", index=False)
df

,location,Population,TotalCases,TotalInfectionRate
0,Faeroe Islands,49053.0,24360.0,49.660571
1,Andorra,77354.0,36808.0,47.583835
2,Gibraltar,33691.0,13740.0,40.782405
3,San Marino,34010.0,13571.0,39.902970
4,Seychelles,98910.0,38120.0,38.540087
...,...,...,...,...
220,Northern Cyprus,0.0,0.0,0.000000
221,Niue,1614.0,0.0,0.000000
222,Nauru,10873.0,0.0,0.000000
223,Jersey,101073.0,0.0,0.000000


**Infection Rate through Time in by Location**

In [8]:
# infection rate through time 
df = h.execute_q('''
    SELECT continent, location, population,
            total_cases, total_deaths,
            (total_cases/population)*100 AS InfectionRate
    FROM CovidDeaths
    WHERE continent IS NOT NULL
    ORDER BY location;
''', cur).fillna(0)
df.to_csv("df_output/infection_rate_by_time.csv", index=False)
df

,continent,location,population,total_cases,total_deaths,InfectionRate
0,Asia,Afghanistan,39835428.0,5.0,0.0,0.000013
1,Asia,Afghanistan,39835428.0,5.0,0.0,0.000013
2,Asia,Afghanistan,39835428.0,5.0,0.0,0.000013
3,Asia,Afghanistan,39835428.0,5.0,0.0,0.000013
4,Asia,Afghanistan,39835428.0,5.0,0.0,0.000013
...,...,...,...,...,...,...
150614,Africa,Zimbabwe,15092171.0,230170.0,5357.0,1.525095
150615,Africa,Zimbabwe,15092171.0,230170.0,5357.0,1.525095
150616,Africa,Zimbabwe,15092171.0,230402.0,5362.0,1.526633
150617,Africa,Zimbabwe,15092171.0,230402.0,5362.0,1.526633


### By Continent

In [9]:
# Total Deaths in Each Continent
h.execute_q('''
    SELECT location, MAX(total_deaths) as TotalDeathCount
    FROM CovidDeaths
    WHERE continent IS NULL 
            AND location NOT LIKE '%income'
            AND location NOT IN ('International', 'World')
    GROUP BY location
    ORDER BY TotalDeathCount DESC;
''', cur)

,location,TotalDeathCount
0,Europe,1643765.0
1,North America,1319863.0
2,Asia,1312025.0
3,South America,1226892.0
4,European Union,969519.0
5,Africa,241578.0
6,Oceania,6730.0


#### Vaccination Rates

In [10]:
# vaccination counts and vaccination rates by continent
h.execute_q('''
    SELECT det.location,
            MAX(det.population) TotalPopulation,
            MAX(vac.total_vaccinations) TotalVaccinations,
            MAX(vac.people_vaccinated/det.population)*100 VaccinatedPercent
    FROM CovidDeaths det
    JOIN CovidVacc vac
        ON vac.iso_code == det.iso_code
        AND vac.date = det.date
    WHERE 
        det.continent IS NULL
        AND det.location NOT LIKE '%income'
        AND det.location NOT IN ('World', 'International')
    GROUP BY det.location
    ORDER BY det.location
''', cur)

,location,TotalPopulation,TotalVaccinations,VaccinatedPercent
0,Africa,1.373486e+09,361600730,16.336897
1,Asia,4.678445e+09,6962875394,71.051091
2,Europe,7.489630e+08,1213996771,67.348292
3,European Union,4.471899e+08,831720628,74.738158
4,North America,5.965813e+08,905668733,69.920492
5,Oceania,4.321995e+07,64683873,64.795687
6,South America,4.342601e+08,731856866,79.741219


In [11]:
# vaccination counts and vaccination rate by country
h.execute_q('''
    SELECT det.location,
            MAX(det.total_cases) TotalCases,
            MAX(det.population) TotalPopulation,
            MAX(vac.total_vaccinations) TotalVaccinations,
            MAX(vac.people_vaccinated/det.population)*100 VaccinatedPercent
    FROM CovidDeaths det
    JOIN CovidVacc vac
        ON vac.iso_code == det.iso_code
        AND vac.date = det.date
    WHERE 
        det.continent IS NOT NULL
        AND det.location NOT LIKE '%income'
        AND det.location NOT IN ('World', 'International')
    GROUP BY det.location
    ORDER BY det.location;
''', cur)

,location,TotalCases,TotalPopulation,TotalVaccinations,VaccinatedPercent
0,Afghanistan,166924.0,39835428.0,5152297.0,11.491630
1,Albania,264875.0,2872934.0,2634377.0,43.695748
2,Algeria,258478.0,44616626.0,12974545.0,16.244588
3,Andorra,36808.0,77354.0,140193.0,74.603770
4,Angola,98409.0,33933611.0,15039557.0,29.468225
...,...,...,...,...,...
220,Vietnam,2358786.0,98168829.0,181665411.0,80.556087
221,Wallis and Futuna,454.0,11094.0,12287.0,55.444384
222,Yemen,11266.0,30490639.0,744060.0,1.930514
223,Zambia,307317.0,18920657.0,2493228.0,4.263124


In [12]:
# cumulative vaccination count by time in EU
h.execute_q('''
    SELECT vac.total_vaccinations
    FROM CovidVacc vac
    WHERE 
        continent IS NOT NULL
        AND location LIKE 'United States';
''', cur)

,total_vaccinations
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
743,542918101.0
744,543382775.0
745,543599880.0
746,543619492.0


In [13]:
# cumulative sum of new vaccinations over time in each country
h.execute_q('''
    SELECT 
        location, date,
        SUM(new_vaccinations) OVER (PARTITION BY location ORDER BY date) cuml_VaccCounts
    FROM CovidVacc
    WHERE
        continent IS NOT NULL;
''', cur)

,location,date,cuml_VaccCounts
0,Afghanistan,2020-02-24,NaN
1,Afghanistan,2020-02-25,NaN
2,Afghanistan,2020-02-26,NaN
3,Afghanistan,2020-02-27,NaN
4,Afghanistan,2020-02-28,NaN
...,...,...,...
150614,Zimbabwe,2022-02-03,6771698.0
150615,Zimbabwe,2022-02-04,6782681.0
150616,Zimbabwe,2022-02-05,6787936.0
150617,Zimbabwe,2022-02-06,6790905.0


In [14]:
# cumulative vaccination percent over time in each country
h.execute_q('''
    WITH VaccRate (location, date, population, cuml_VaccCounts) AS
    (
        SELECT 
            vac.location, vac.date, det.population,
            SUM(vac.new_vaccinations) OVER (PARTITION BY vac.location ORDER BY vac.date) cuml_VaccinationCounts
        FROM CovidVacc vac
        JOIN CovidDeaths det 
            ON det.iso_code = vac.iso_code
            AND det.date = vac.date
        WHERE
            vac.continent IS NOT NULL
    )
    SELECT *, (cuml_VaccCounts/population)*100 cuml_VaccPercent
    FROM VaccRate;
''', cur)

,location,date,population,cuml_VaccCounts,cuml_VaccPercent
0,Afghanistan,2020-02-24,39835428.0,NaN,NaN
1,Afghanistan,2020-02-25,39835428.0,NaN,NaN
2,Afghanistan,2020-02-26,39835428.0,NaN,NaN
3,Afghanistan,2020-02-27,39835428.0,NaN,NaN
4,Afghanistan,2020-02-28,39835428.0,NaN,NaN
...,...,...,...,...,...
150614,Zimbabwe,2022-02-03,15092171.0,6771698.0,44.868946
150615,Zimbabwe,2022-02-04,15092171.0,6782681.0,44.941718
150616,Zimbabwe,2022-02-05,15092171.0,6787936.0,44.976538
150617,Zimbabwe,2022-02-06,15092171.0,6790905.0,44.996210


In [15]:
# Create view of cumulative sum of new vaccination count over time in each country
h.execute_q('''DROP VIEW IF EXISTS cuml_VaccCount;''', cur)

h.execute_q('''
    CREATE VIEW cuml_VaccCount AS
    SELECT 
        location, date,
        SUM(new_vaccinations) OVER (PARTITION BY location ORDER BY date) cuml_VaccCounts
    FROM CovidVacc
    WHERE
        continent IS NOT NULL;
''', cur)

""


### Close the connection

In [16]:
# close the connection
con.close()